# Numerical Analysis theory

We have seen a range of numerical methods, and through their implementations seen some of their properties. Whilst some bits of theory are easy to state ("high order good, low order bad"), some require a little more calculation to see clearly.

## Discrete dispersion relations

We noted the CFL limit, that the discrete timestep is limited by the grid spacing and the maximum propagation speed. However, we did not show how to calculate it. Nor did we investigate other aspects of the error. Both can be done - in simple cases! - through dispersion relations.

We will focus on the advection equation
$$
\partial_t q + \partial_x (v q) = 0
$$
and use either the method of lines or the fully discrete method where time is approximated using forward differences and space using backward difference. In this latter case the discrete algorithm is
$$
q^{n+1}_j = q^n_j - \frac{v \Delta t}{\Delta x} (q^n_j - q^n_{j-1}) \, ,
$$
and we will write $\sigma = v \Delta t / \Delta x$ for the *convection number*.

We remember that in a dispersion relation we look at how a plane wave propagates, by looking for solutions of the form $q(x, t) = \exp(i k x - i \omega t)$. We assume the wavenumber $k$ is real and known, and find the appropriate value of $\omega$, which is generally complex. If $\omega$ has positive imaginary part then the solution is unstable. Substituting this into the continuum advection equation gives
$$
\omega = v k \, .
$$
The solution is purely real, and both the group velocity $\partial_k \omega$ and the phase velocity $\omega / k$ are equal to the advection velocity $v$.



### Von Neumann stability



We want to know how the discrete solution behaves, so we substitute in the same ansatz into the discrete algorithm. We find
$$
\exp(-i \omega \Delta t) = 1 - \sigma (1 - \exp(-i k \Delta x)) \, .
$$
As $k$ is real we see that
$$
\begin{aligned}
\exp(\mathrm{Im}(\omega) \Delta t) \cos(\mathrm{Re}(\omega) \Delta t) &= 1 - \sigma (1 - \cos(k \Delta x)) \, , \\
\exp(\mathrm{Im}(\omega) \Delta t) \sin(\mathrm{Re}(\omega) \Delta t) &= \sigma \sin(k \Delta x) \, .
\end{aligned}
$$
We now use the notation $A = \exp(\mathrm{Im}(\omega) \Delta t)$. This is the *amplification factor*; the amount that a single Fourier mode (with wavenumber $k$) grows by in a single timestep. It follows that
$$
\begin{aligned}
A^2 &= \left( (1 - \sigma) + \sigma \cos(k \Delta x) \right)^2 + \sigma^2 \sin^2(k \Delta x) \\
&= (1 - \sigma)^2 + 2 \sigma (1 - \sigma) \cos(k \Delta x) + \sigma^2 \, .
\end{aligned}
$$

The limiting cases occur when $\cos(k \Delta x) = \pm 1$. When it is $+1$ we have
$$
A^2 = 1 .
$$
The solution neither grows nor decays. However, when the cosine term is $-1$ we have
$$
A^2 = 1 - 4 \sigma + 4 \sigma^2 = (1 - 2 \sigma)^2 \, .
$$
When $\sigma > 1$ this gives the amplification factor $A > 1$ and hence the solution grows at each timestep, showing an instability. This is precisely the CFL condition.

This set of steps is *Von Neumann stability analysis*. It works for linear problems and allows us to look at stability, as with CFL. It also allows us to talk about the *damping* in the solution. We see that the amplitude of the solution will decrease with each timestep by a factor of $A$, which depends on the wavenumber $k$. We can plot this directly.






In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipympl

In [ ]:
kdx = np.linspace(1e-10, np.pi-1e-10, 100)
plt.figure()
for sigma in [0.6, 0.75, 0.9]:
    A = np.sqrt(((1-sigma)**2 + 2*sigma*(1-sigma)*np.cos(kdx) + sigma**2))
    plt.plot(kdx, A, label=rf'$\sigma={sigma}$')
plt.legend()
plt.xlim(0, np.pi)
plt.xticks([0, np.pi/2, np.pi], ['0', r'$\pi/2$', r'$\pi$'])
plt.xlabel(r'$k \Delta x$')
plt.ylabel('$A$')
plt.legend()
plt.show()

We see how the low frequency modes (on the grid) are well captured, but the high frequency modes are damped. This is a general feature of numerical methods. We also see that the closer the timestep is to the stability limit, the less damping there is.

### Phase errors

The true solution to the dispersion relation for the advection equation is $\omega = v k$. Using the results above we see that the discrete solution is
$$
\tan(\mathrm{Re}(\omega) \Delta t) = \frac{\sigma \sin(k \Delta x)}{1 - \sigma + \sigma \cos(k \Delta x)} \, .
$$
Using $\sigma = v \Delta t / \Delta x$ we can write this as
$$
\frac{\mathrm{Re}(\omega)}{k} = \frac{v}{\sigma k \Delta x}  \arctan  \left\{ \frac{\sigma \sin(k \Delta x)}{1 - \sigma + \sigma \cos(k \Delta x)} \right\} \, .
$$
Again, we can plot this directly.

In [ ]:
v = 1
plt.figure()
for sigma in [0.6, 0.75, 0.9]:
    re_om_k = v / (sigma * kdx) * np.arctan2(sigma*np.sin(kdx), 1-sigma+sigma*np.cos(kdx))
    plt.plot(kdx, re_om_k, label=rf'$\sigma={sigma}$')
plt.legend()
plt.xlim(0, np.pi)
plt.xlabel(r'$k \Delta x$')
plt.ylabel(r'$\frac{\mathrm{Re}(\omega)}{k}$')
plt.xticks([0, np.pi/2, np.pi], ['0', r'$\pi/2$', r'$\pi$'])
plt.show()

Again we see that the numerical phase speed is correct at low wavenumbers and improves as the timestep approaches the stability limit. However, high wavenumbers travel *faster* than they should. This is *not* a general feature, even of this method! Let us check what happens with very small timesteps:

In [ ]:
plt.figure()
for sigma in [0.1, 0.25, 0.4]:
    re_om_k = v / (sigma * kdx) * np.arctan2(sigma*np.sin(kdx), 1-sigma+sigma*np.cos(kdx))
    plt.plot(kdx, re_om_k, label=rf'$\sigma={sigma}$')
plt.legend()
plt.xlim(0, np.pi)
plt.xlabel(r'$k \Delta x$')
plt.ylabel(r'$\frac{\mathrm{Re}(\omega)}{k}$')
plt.xticks([0, np.pi/2, np.pi], ['0', r'$\pi/2$', r'$\pi$'])
plt.show()

We see that when the timestep is less than half the stability limit, the higher wavenumbers propagate *slower* than they should. In the limit, grid frequency modes (where $k \Delta x \to \pi$) do not propagate at all.

The differing propagation speeds with wavenumber is typically referred to as *dispersion error*. Particularly for gravitational waves it is more concerning than the damping error, particularly for producing template waveforms.

### Points per wavelength

As nonlinear simulations are so expensive, we would like to know before starting how many grid points give us "good enough" accuracy. In general the only answer is to try low resolutions that you can easily afford and measure the error. However, a rule of thumb can be found from very simplified calculations.

For the advection (or wave) equation, for linear schemes, we can look at how a single Fourier mode behaves. Denote the exact solution as $q(x, t) = \exp(i k x - i \omega t)$, and the numerical solution as $q_{\text{scheme}, \Delta x}(x, t) = \exp(i k x - i \omega_{\text{scheme}, \Delta x} t)$. Then the relative error is
$$
\mathcal{E}_{\text{scheme}, \Delta x} = \left| \frac{q_{\text{scheme}, \Delta x} - q}{q} \right|
$$
which can be approximated, at a fixed time $t=T$, as
$$
\mathcal{E}_{\text{scheme}, \Delta x} \simeq \left| (\omega - \omega_{\text{scheme}, \Delta x}) T \right| \, .
$$
As we know that the phase velocity is $v = \omega / k$, we can re-write this as
$$
\mathcal{E}_{\text{scheme}, \Delta x} \simeq \left| k (v - v_{\text{scheme}, \Delta x}) T \right| \, .
$$



Using the Method of Lines we can typically use methods that minimize the error in time, so we focus on the spatial error. For example, using central differences we have
$$
\begin{aligned}
&&\partial_t q_{\text{2cd}, \Delta x} &= - i \omega_{\text{2cd}, \Delta x} q \\
&&&= -v \partial_x q_{\text{2cd}, \Delta x} \\
&&&\simeq -\frac{v q_{\text{2cd}, \Delta x}}{2 \Delta x} \left( \exp(i k \Delta x) - \exp(-i k \Delta x) \right) \\
&&&= -\frac{i v q_{\text{2cd}, \Delta x}}{\Delta x} \sin(k \Delta x) \\
\implies && \omega_{\text{2cd}, \Delta x} &= \frac{v}{\Delta x} \sin(k \Delta x) \\
\implies && v_{\text{2cd}, \Delta x} &= v \frac{\sin(k \Delta x)}{k \Delta x} \, .
\end{aligned}
$$

Therefore the phase error is
$$
\begin{aligned}
\mathcal{E}_{\text{2cd}, \Delta x} & \simeq \left| k v \left(1 - \frac{\sin(k \Delta x)}{k \Delta x} \right) T \right| \\
&\simeq k v T \frac{(k \Delta x)^2}{6} \, .
\end{aligned}
$$


Finally, we want to non-dimensionalize the result. We define the number of *points per wavelength* as
$$
p = \frac{2 \pi}{k \Delta x} \, ,
$$
as we are working on a domain of length $L = 2 \pi$, and the number of *evolution periods* as
$$
\nu = \frac{k v T}{2 \pi} \, ,
$$
in order to get the wave travel time to time $T$. We can then write the error as
$$
\mathcal{E}_{\text{2cd}, \Delta x} \simeq \frac{\pi \nu}{3} \left(\frac{2 \pi}{p} \right)^2 \, .
$$

We re-arrange this to give us the actual useful quantity:
$$
p_{\text{2cd}} \gtrsim 2 \pi \sqrt{\frac{\pi \nu}{3 \mathcal{E}_{\text{2cd}, \Delta x}}} \, .
$$
This is our rule of thumb: the number of points per wavelength required scales as the square root of the number of evolution periods, divided by the (relative) error desired.

Check one standard result: the number of points needed for a 10% error over a single evolution period:

In [ ]:
def p2cd(nu, e2cd):
    return 2*np.pi*np.sqrt(np.pi*nu/(3*e2cd))

e2cd = 0.1
nu = 1
print(f"Points needed for {e2cd} error over {nu} evolution periods is {p2cd(nu, e2cd):.1f}")

The fundamental mode of a neutron star is around 3kHz. Assume we want to solve for 10ms, so that $\nu = 30$. Check how many points per wavelength are needed for a 1% error. Assuming a fundamental mode wavelength of around the size of the neutron star, so $\sim 25$ km, check that the required grid spacing is $\sim 70$ m. 

In [ ]:
e2cd = 0.01
nu = 30
p = p2cd(nu, e2cd)
print(f"Points needed for {e2cd} error over {nu} evolution periods is {p:.1f}")
L = 25000
dx = L/p
print(f"Assume L of 25km, so dx <= {dx:.2f}m")

Repeat the derivation for fourth order central differencing, where
$$
\partial_x q \simeq \frac{1}{12 \Delta x} \left( -q_{j+2} + 8 q_{j+1} - 8 q_{j-1} + q_{j-2} \right) \, .
$$
Show that
$$
v_{\text{4cd}, \Delta x} = v \frac{8 \sin(k \Delta x) - \sin(2 k \Delta x)}{6 k \Delta x} \, .
$$
Use this to show that
$$
p_{\text{4cd}} \gtrsim 2 \pi \sqrt[4]{\frac{\pi \nu}{15 \mathcal{E}_{\text{4cd}, \Delta x}}} \, .
$$
Check that this means resolving the fundamental mode of a neutron star for 10ms to 1% phase error requires a grid spacing of $\sim 800$ m.

In [ ]:
def p4cd(nu, e4cd):
    return 2*np.pi*(np.pi*nu/(15*e4cd))**(0.25)

e4cd = 0.01
nu = 30
p = p4cd(nu, e4cd)
print(f"Points needed for {e4cd} error over {nu} evolution periods is {p:.1f}")
L = 25000
dx = L/p
print(f"Assume L of 25km, so dx <= {dx:.2f}m")

Check your other favourite modes and required evolution times.